<a href='http://www.holoviews.org'><img src="../../assets/pn_hv_gv_bk_ds_pa.png" alt="HoloViz logos" width="40%;" align="left"/></a>
<div style="float:right;"><h2>Exercise 4: Building a Dashboard</h2></div>

In [ ]:
import numpy as np
import pandas as pd
import panel as pn
pn.extension('katex')

import hvplot.pandas # noqa

## Non-interactive components

In this exercise you will first construct a number of Panel components and then lay them out as a non-interactive Panel dashboard, using the usual earthquakes dataset.

In [ ]:
%%time
df = pd.read_parquet('../../data/earthquakes-projected.parq')
df.time = df.time.astype('datetime64[ns]')
df = df.set_index(df.time)
most_severe = df[df.mag >= 7]

#### The logo

The first component of the dashboard is an image of the US Geological Survey logo. Start by declaring a pane containing the logo and assign it to the ``logo`` variable. Also set a width to ensure the logo doesn't appear too big.

In [ ]:
logo_url = '../../assets/usgs_logo.png'

## Define a panel component containing the logo
logo = ...

## Display it


<details><summary>Solution</summary><br>

```python
logo = pn.panel(logo_url, width=300) 
```

</details>

#### Richter scale equation

Next we will create a component to display the equation for the Richter scale definition. Declare the appropriate pane and assign it to the ``equation`` variable.

In [ ]:
equation_string = '$M_L = log_{10}A - log_{10} A_0(\delta)$'

## Define a panel component containing the equation (Hint: Use the LaTeX pane)
equation = ...

## Display it


<details><summary>Solution</summary><br>

```python
equation = pn.pane.LaTeX(equation_string) 
```

</details>

#### List the strongest earthquakes

In [ ]:
year = 2000

def strongest_earthquakes_fn(year):
    year_df = df[(df.time.dt.year == year) & (df.mag > 7)]
    return year_df.sort_values('mag', ascending=False).iloc[:5][['time', 'place', 'mag']].reset_index(drop=True)

## Create a panel component by calling the function with a particular year
strongest_earthquakes = ...

## Display it


<details><summary>Solution</summary><br>

```python
strongest_earthquakes = pn.panel(strongest_earthquakes_fn(year)) 
```

</details>

#### Display an iframe of a Google Map

<details><summary>Hint</summary><br>

An iframe is an HTML tag.

</details>

In [ ]:
def gmap_fn(year):
    yearly_df = df[(df.time.dt.year == year)]
    index = np.argmax(yearly_df.mag.fillna(0).values)
    strongest = yearly_df.iloc[index]
    lon, lat = strongest.longitude, strongest.latitude
    return """
    <iframe width="300" height="300" src="https://maps.google.com/maps?q={lat},{lon}&z=6&output=embed"
    frameborder="0" scrolling="no" marginheight="0" marginwidth="0"></iframe>
    """.format(lat=lat, lon=lon)

## Create a panel component by calling the function with a particular year and wrapping it in the appropriate pane
gmap = ...

## Display it


<details><summary>Solution</summary><br>

This example solution displays the map for the `year` (by default is the year 2000):

```python
gmap = pn.panel(gmap_fn(year))
```

</details>

#### A plot 

Here we create a plot from the ``year_df`` dataframe with `.hvplot`, create a Panel component, and assign it to the ``plot`` variable. We can visualize the distribution of earthquake magnitudes over the year by plotting the `time` variable on the `x`-axis and the `mag` variable on the `y`-axis.



In [ ]:
year_df = df[df.time.dt.year == year]

## Create a plot and assign it to the plot variable
plot = ...

## Display it


<details><summary>Solution</summary><br>

This example solution uses concepts covered in the plotting section of the tutorial:

```python
plot = year_df.hvplot.line(x='time', y='mag')
```

</details>

### Composing the static dashboard

Now that we have defined all the different components, it is time to lay them out into the overall dashboard.

Arrange the following components into a dashboard using the ``Row`` and ``Column`` panels:

* ``logo``
* ``equation``
* ``strongest_earthquakes``
* ``gmap``
* ``plot``


<details><summary>Hint</summary><br>

Remember that you can nest panel `Row` and `Column` layouts and use `HSpacer` and `VSpacer` objects to adjust spacing as necessary.

</details>

In [ ]:
# Static Dashboard combining all the elements above.

<details><summary>Example Solution</summary><br>

```python
year = 2000
logo_url = '../../assets/usgs_logo.png'
equation_string = '$M_L = log_{10}A - log_{10} A_0(\delta)$'

def strongest_earthquakes_fn(year):
    year_df = df[(df.time.dt.year == year) & (df.mag > 7)]
    return year_df.sort_values('mag', ascending=False).iloc[:5][['time', 'place', 'mag']].reset_index(drop=True)

def gmap_fn(year):
    yearly_df = df[(df.time.dt.year == year)].fillna(0)
    index = np.argmax(yearly_df.mag.fillna(0).values)
    strongest = yearly_df.iloc[index]
    lon, lat = strongest.longitude, strongest.latitude
    return """
    <iframe width="300" height="300" src="https://maps.google.com/maps?q={lat},{lon}&z=6&output=embed"
    frameborder="0" scrolling="no" marginheight="0" marginwidth="0"></iframe>
    """.format(lat=lat, lon=lon)

logo = pn.panel(logo_url, width=200)
equation = pn.pane.LaTeX(equation_string)
strongest_earthquakes = strongest_earthquakes_fn(year)
gmap = pn.pane.HTML(gmap_fn(year), height=300, width=300)
year_df = df[(df.time.dt.year == year) & (df.mag > 7)]
plot = year_df.hvplot.line(x='time', y='mag')

title = pn.panel('# Strongest Earthquakes in the Year %d' % year, width=400)
header = pn.Row(title, pn.layout.HSpacer(), logo)

body_row1 = pn.Row(
    pn.Column('### Richter scale definition', equation, '### Strongest Earthquake', gmap),
    pn.Column('### Strongest Earthquakes', strongest_earthquakes),
)

pn.Column(header, pn.Column(body_row1,pn.Column('### Magnitude Plot', plot)))
```
    
</details>

### Exercise 3

Having learned about how to create interactive components, we can now integrate them into dashboards. Here is one of the interactive plots from the last exercise:

In [ ]:
depth_slider = pn.widgets.FloatSlider(name='Minimum depth', start=0, end=700, value=350)

dfi = most_severe.interactive(sizing_mode='stretch_width')
depth_filtered = dfi[dfi['depth'] < depth_slider]
depth_filtered_plot = depth_filtered.hvplot(y='mag', kind='scatter', color='red', marker='x')

Have a look at this object to remind you what it represents.

#### Adding a title and description

Using the following markdown description of this plot, create a dashboard that integrates it together with an appropriate title, USGS logo and the interactive plot itself.

In [ ]:
plot_description_markdown = """
This plot filters earthquakes according to the minimum depth
slider and displays the magnitude of the filtered earthquakes 
over time. Note that only earthquakes with a magnitude `>7` 
are shown.
"""

<details><summary>Solution</summary><br>

```python
year_slider = pn.widgets.IntSlider(name='Year', start=2000, end=2018, value=2000)
year_slider
```

</details>

In [ ]:
title = pn.panel('# Earthquake magnitude as function of depth')
header = pn.Row(title, pn.layout.HSpacer(), logo)

pn.Column(header, pn.panel(plot_description_markdown), depth_filtered_plot)
